In [ ]:
import pandas as pd
# import matplotlib.pyplot as plt
import bqplot.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML
from tables import *

In [ ]:
DATA_URL = 'https://gist.githubusercontent.com/chriddyp/cb5392c35661370d95f300086accea51/raw/8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/indicators.csv'

# Add a title or something

## Add a subheading

Add some text for Voila to render. 

In [ ]:
h5file_data = open_file('all_data.h5', 'r')

In [ ]:
class App:
    
    def __init__(self, df):
        
        self._df = df
        available_indicators = self._df['Indicator Name'].unique()
        self._x_dropdown = self._create_indicator_dropdown(available_indicators, 0)
        self._y_dropdown = self._create_indicator_dropdown(available_indicators, 1)        
        self._plot_container = widgets.Output()
        self._year_slider, year_slider_box = self._create_year_slider(
            min(df['Year']), max(df['Year'])
        )
        # Add the variable parameter sliders. 
        self._potential_height_slider, potential_height_slider_box = self._create_parameter_slider(
            min_val=0.0, max_val=0.5, step_val=0.10, default_val=0.0, model_param=r'$V_H$')
        self._tp_slider, tp_slider_box = self._create_parameter_slider(
            min_val=-0.5, max_val=0.5, step_val=0.05, default_val=-0.05, model_param=r"$t'$")
        self._tsoc_slider, tsoc_slider_box = self._create_parameter_slider(
            min_val=-0.2, max_val=0.2, step_val=0.02, default_val=0.0, model_param=r"$t_{soc}$")
        self._deltaU1_slider, deltaU1_slider_box = self._create_parameter_slider(
            min_val=-0.5, max_val=0.5, step_val=0.1, default_val=0.0, model_param=r"$\Delta U_1$")
        self._deltaU2_slider, deltaU2_slider_box = self._create_parameter_slider(
            min_val=-0.5, max_val=0.5, step_val=0.1, default_val=0.0, model_param=r"$\Delta U_2$")
        

        self.container = widgets.VBox([
            self._plot_container,
            potential_height_slider_box, 
            tp_slider_box,
            tsoc_slider_box,
            deltaU1_slider_box,
            deltaU2_slider_box
        ])
        
        self._update_app()
    
    
    @classmethod
    def from_url(cls, url):
        df = pd.read_csv(url)
        return cls(df)
        
    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    def _create_year_slider(self, min_year, max_year):
        year_slider_label = widgets.Label('Year range: ')
        year_slider = widgets.IntRangeSlider(
            min=min_year, max=max_year,
            layout=widgets.Layout(width='500px')
        )
        year_slider.observe(self._on_change, names=['value'])
        year_slider_box = widgets.HBox([year_slider_label, year_slider])
        return year_slider, year_slider_box
    
    def _create_parameter_slider(self, min_val, max_val, step_val, default_val, model_param):
        parameter_slider = widgets.FloatSlider(
            min=min_val, max=max_val, step=step_val, value=default_val, 
            description=model_param+': ', layout=widgets.Layout(width='500px')
        )
        parameter_slider.observe(self._on_change, names=['value'])
        parameter_slider_box = widgets.HBox([parameter_slider])
        return parameter_slider, parameter_slider_box
    
    def _get_bands(self, potential_height, tp, tsoc, deltaU1, deltaU2):
        # Nearest-neighbour hopping parameter fixed. 
        t = -1.0
        # For now keep potential_period fixed.
        potential_period = 2.0
        params_path_potential = 'VL' + '{0:.0f}'.format(potential_period) + '_VH' + '{0:.3f}'.format(potential_height)
        params_path_hoppings = '_t' + '{0:.2f}'.format(t) + '_tp' + '{0:.2f}'.format(tp) + \
                               '_tsoc' + '{0:.2f}'.format(tsoc)
        params_path_onsite = '_deltaU1' + '{0:.2f}'.format(deltaU1) + '_deltaU2' + '{0:.2f}'.format(deltaU2)
        bands_filename = params_path_potential + params_path_hoppings + params_path_onsite
        bands = h5file_data.get_node('/VL2_data', 'bands_'+bands_filename)
        return bands
    
    def _create_plot(self, x_indicator, y_indicator, year_range, potential_height_value, 
                             tp_value, tsoc_value, deltaU1_value, deltaU2_value):
        df = self._df[self._df['Year'].between(*year_range)]
        xs = df[df['Indicator Name'] == x_indicator]['Value']
        ys = df[df['Indicator Name'] == y_indicator]['Value']
        # plt.figure(figsize=(8, 6))
        # plt.xlabel(x_indicator, size=16)
        # plt.ylabel(y_indicator, size=16)
        # plt.gca().tick_params(axis='both', which='major', labelsize=16)
        # plt.plot(xs, ys, 'o', alpha=0.3)
        
#         plt.figure()
#         x = np.linspace(-5, 5, 100)
#         y = tp_value*x**3 - 5*tsoc_value*x**2 + 10*deltaU1_value*x - 50*deltaU2_value
#         for factor in range(1, 5):
#             plt.plot(x, factor*y)
        
        kpoints = h5file_data.get_node('/VL2_data', 'k_points')
        bands = self._get_bands(potential_height_value, tp_value, tsoc_value, 
                                deltaU1_value, deltaU2_value)
        plt.figure()
        for band_ind in range(len(bands[0])):
            plt.plot(kpoints[:], bands[:, band_ind])
        plt.xlabel(r'$k$ path')
        plt.ylabel(r'$E$ (eV)')
        
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        x_indicator = self._x_dropdown.value
        y_indicator = self._y_dropdown.value
        year_range = self._year_slider.value
        potential_height_value = self._potential_height_slider.value
        tp_value = self._tp_slider.value
        tsoc_value = self._tsoc_slider.value
        deltaU1_value = self._deltaU1_slider.value
        deltaU2_value = self._deltaU2_slider.value
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._create_plot(x_indicator, y_indicator, year_range, potential_height_value, 
                             tp_value, tsoc_value, deltaU1_value, deltaU2_value)
            plt.show()

In [ ]:
app = App.from_url(DATA_URL)

app.container